In [1]:
import getpass
import os
from langchain_core.tools import tool

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [23]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [8]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [9]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [12]:
llm_with_tools = llm.bind_tools([multiply])

In [30]:
msg = llm_with_tools.invoke("whats 5 times 55")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 55},
  'id': 'call_ur0FSF6CEr6vC9570qUjAbog'}]

In [32]:
# from operator import itemgetter
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 10")

40

In [34]:
llm.invoke("whats 5 times forty two")

AIMessage(content='5 times 42 is equal to 210.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-85428222-6d50-464d-9dd3-cc64bf15f09a-0')

In [35]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [99]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [106]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [112]:
prompt_str = """================================ System Message ================================

You are a helpful assistant

============================= Intermediate Steps =============================

{intermediate_steps}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}"""

In [117]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(template=prompt_str, input_variables=['intermediate_steps', 'agent_scratchpad', 'input'])

In [118]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [119]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [120]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `multiply` with `{'first_int': 3645, 'second_int': 3645}`


13286025
Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `multiply` with `{'first_int': 13286025, 'second_int': 13286025}`


176518460300625
Invoking: `multiply` with `{'first_int': 176518460300625, 'second_int': 176518460300625}`


31158766826903324165375390625To solve the expression "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result":

1. Calculate 3 to the fifth power:
   \( 3^5 = 243 \)

2. Calculate the sum of twelve and three:
   \( 12 + 3 = 15 \)

3. Multiply the result from step 1 by the result from step 2:
   \( 243 \times 15 = 3645 \)

4. Square the result from step 3:
   \( 3645^2 = 13,286

{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'To solve the expression "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result":\n\n1. Calculate 3 to the fifth power:\n   \\( 3^5 = 243 \\)\n\n2. Calculate the sum of twelve and three:\n   \\( 12 + 3 = 15 \\)\n\n3. Multiply the result from step 1 by the result from step 2:\n   \\( 243 \\times 15 = 3645 \\)\n\n4. Square the result from step 3:\n   \\( 3645^2 = 13,286,025 \\)\n\n5. Finally, square the result from step 4:\n   \\( 13,286,025^2 = 176,518,460,300,625 \\)\n\nTherefore, the final result after following the steps is \\( 176,518,460,300,625 \\).'}

In [121]:
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

tools = [multiply, exponentiate, add]
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools

In [123]:
chain.invoke("What's 23 times 7")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1Y0epSX0cG9CApFrlRFSW2so', 'function': {'arguments': '{"first_int":23,"second_int":7}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 149, 'total_tokens': 168}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f5b1f08b-d826-4bf0-9e78-5b92ceb27870-0', tool_calls=[{'name': 'multiply', 'args': {'first_int': 23, 'second_int': 7}, 'id': 'call_1Y0epSX0cG9CApFrlRFSW2so'}])

In [130]:
def tool_chain(model_output):
      tool_map = {tool.name: tool for tool in tools}    
      chosen_tool = tool_map[model_output["name"]]
      return itemgetter("arguments") | chosen_tool.invoke()

model_output = {'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}
print(tool_chain(model_output).invoke())

ValidationError: 2 validation errors for multiplySchema
first_int
  value is not a valid integer (type=type_error.integer)
second_int
  field required (type=value_error.missing)

In [131]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [132]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [134]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [136]:
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun

wikidata = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())

In [139]:
tools = [wikidata]
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [152]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [141]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [142]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
I should use Wikidata to find information about LangChain.
Action: Wikidata
Action Input: LangChainResult Q117340550:
Label: LangChain
Description: language model application development framework
instance of: software
founded by: Harrison Chase

Result Q134219:
Label: polysaccharide
Description: long chain of units of monosaccharide (carbohydrate)
Aliases: polysaccharides
instance of: structural class of chemical entities
subclass of: carbohydrate, macromolecule, biopolymer
part of: polysaccharide immune receptor activity, polysaccharide binding, polysaccharide biosynthetic process, polysaccharide catabolic process, polysaccharide metabolic process, glycolysis from storage polysaccharide through glucose-1-phosphate, polysaccharide localization, polysaccharide transmembrane transporter activity, polysaccharide transport
has part(s): carbon, hydrogen, oxygenThe information I need is under the result Q117340550.
Final Answer: LangChain is a languag

{'input': 'what is LangChain?',
 'output': 'LangChain is a language model application development framework founded by Harrison Chase.'}

In [148]:
wikidata.args

{'query': {'title': 'Query', 'type': 'string'}}

In [153]:
prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [154]:
from crewai_tools import WebsiteSearchTool

In [155]:
tool = WebsiteSearchTool()

In [156]:
tool = WebsiteSearchTool(website='https://accessibe.com')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


In [157]:
tool

WebsiteSearchTool(name='Search in a specific website', description="Search in a specific website(search_query: 'string') - A tool that can be used to semantic search a query from https://accessibe.com website content.", args_schema=<class 'crewai_tools.tools.website_search.website_search_tool.FixedWebsiteSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x75692cf53380>, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x75692c87f7d0>, summarize=False), config=None)